<a href="https://colab.research.google.com/github/Ananyas-7/ai-dyslexia-tool/blob/main/train_simplifier.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install -q transformers datasets sentencepiece accelerate
from datasets import load_dataset
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, DataCollatorForSeq2Seq
from transformers import Seq2SeqTrainer, Seq2SeqTrainingArguments
# Choose model
model_name = "t5-small"
tokenizer = AutoTokenizer.from_pretrained(model_name)

In [3]:
!pip install sacrebleu easse textstat

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.8/51.8 kB 3.8 MB/s eta 0:00:00
ERROR: Could not find a version that satisfies the requirement easse (from versions: none)
ERROR: No matching distribution found for easse


In [4]:
import pandas as pd
train = pd.read_csv("train.csv")
valid = pd.read_csv("valid.csv")
print(train.head())
print(valid.head())

          file                                           sentence  \
0  cesa109.pdf                  He has not finished his homework.   
1  aemr104.pdf                              • Hide under a table.   
2  aemr102.pdf                Mama sparrow laid three small eggs.   
3  cesa101.pdf  Unit 1 Fun with Friends PPiiccttuurree rreeaad...   
4  cesa107.pdf  3 4 5 Santoor │ Class 3 69 Reprint 2025-26 UUn...   

                                    complex_sentence  \
0                  He has not finished his homework.   
1                              • Hide under a table.   
2                Mama sparrow laid three small eggs.   
3  Unit 1 Fun with Friends PPiiccttuurree rreeaad...   
4  3 4 5 Santoor │ Class 3 69 Reprint 2025-26 UUn...   

                                              tokens  simple_sentence  
0                               ha finished homework              NaN  
1                                         hide table              NaN  
2                  mama sparrow 

In [18]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [19]:
train_src = "drive/MyDrive/ai-dyslexia-tool/data/raw/wikilarge/data-simplification/wikilarge/wiki.full.aner.ori.train.src"
train_dst = "drive/MyDrive/ai-dyslexia-tool/data/raw/wikilarge/data-simplification/wikilarge/wiki.full.aner.ori.train.dst"

valid_src = "drive/MyDrive/ai-dyslexia-tool/data/raw/wikilarge/data-simplification/wikilarge/wiki.full.aner.ori.valid.src"
valid_dst = "drive/MyDrive/ai-dyslexia-tool/data/raw/wikilarge/data-simplification/wikilarge/wiki.full.aner.ori.valid.dst"

test_src = "drive/MyDrive/ai-dyslexia-tool/data/raw/wikilarge/data-simplification/wikilarge/wiki.full.aner.ori.test.src"
test_dst = "drive/MyDrive/ai-dyslexia-tool/data/raw/wikilarge/data-simplification/wikilarge/wiki.full.aner.ori.test.dst"

In [ ]:
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

# Training setup
training_args = Seq2SeqTrainingArguments(
    output_dir="models/t5_simplifier",
    evaluation_strategy="steps",
    save_strategy="steps",
    learning_rate=3e-4,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=2,
    save_total_limit=1,
    predict_with_generate=True,
    logging_steps=50,
    fp16=True,
    gradient_accumulation_steps=2,
    eval_steps=500,
    save_steps=500,
)

trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=train_ds,
    eval_dataset=valid_ds,
    tokenizer=tokenizer,
    data_collator=data_collator,
)

# Train
trainer.train()

In [ ]:
import pandas as pd
test = pd.read_csv("test.csv")
from datasets import Dataset
test_ds = Dataset.from_pandas(test)


In [ ]:
from transformers import pipeline

# Load the fine-tuned model
model_dir = "models/t5_simplifier"
simplifier = pipeline("text2text-generation", model=model_dir, tokenizer=model_dir)
preds = []
for src in test["src"]:
    output = simplifier("simplify: " + src, max_length=128, num_beams=4, clean_up_tokenization_spaces=True)[0]['generated_text']
    preds.append(output)
test["pred"] = preds
test.head()
